In [ ]:
import pdfplumber
import pdfminer
import requests
import io
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from collections import Counter
import operator

In [ ]:
r = requests.get(pdfurl, stream=True)
f = io.BytesIO(r.content)
# set up pdfminer
rsrcmgr = PDFResourceManager()
retstr = io.StringIO()
codec = 'utf-8'
laparams = LAParams()
device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
pagenos=set()
# extract all text
content = []
for page in PDFPage.get_pages(fp=f, pagenos=pagenos, maxpages=0, caching=True, check_extractable=True):
    interpreter.process_page(page)
text = retstr.getvalue()
content.append(text)
# close apps
device.close()
retstr.close()

In [ ]:
link = "https://arxiv.org/pdf/2002.07646.pdf"

r = requests.get(link, stream=True)
f = io.BytesIO(r.content)

In [ ]:
data = pdfplumber.load(f)

In [ ]:
data.metadata

In [ ]:
fontsize = []
for page in data.pages:
    for c in page.chars:
        if c.get('size') not in fontsize:
            fontsize.append(c.get('size'))

In [ ]:
sizes = dict(Counter(fontsize))

In [ ]:
size = max(sizes.items(), key=operator.itemgetter(1))[0]

In [ ]:
text = ""
for page in data.pages:
    for char in page.chars:
        if char.get('upright') == 1:
            #print(char.get('size'), char.get('upright'), char.get('text'))
            text += char.get('text')

In [ ]:
link = "https://arxiv.org/pdf/2002.06770.pdf"

data.metadata

In [ ]:
link = "https://arxiv.org/pdf/2002.06775.pdf"

data.metadata

In [ ]:
data.metadata

In [ ]:
link = "https://arxiv.org/pdf/2002.07701.pdf"

data.metadata

In [ ]:
thing = []
for i in data.pages[0].chars:
    if i.get('upright') == 0:
        print(i, "\n")

In [ ]:
data.metadata

In [ ]:
data.metadata

In [ ]:
data.metadata

In [ ]:
data.metadata

In [110]:
import pdfquery
import re
from textblob import TextBlob
import requests
import io
import string

In [2]:
link = "https://arxiv.org/pdf/2002.07797.pdf"

r = requests.get(link, stream=True)
f = io.BytesIO(r.content)

In [3]:
data = pdfquery.PDFQuery(f)

In [4]:
data.load()

In [5]:
data.doc.catalog

{'Type': /'Catalog',
 'Names': <PDFObjRef:783>,
 'OpenAction': <PDFObjRef:54>,
 'Outlines': <PDFObjRef:752>,
 'PageMode': /'UseOutlines',
 'Pages': <PDFObjRef:751>}

In [6]:
#data.tree.write("test2.xml", pretty_print=True, encoding="utf-8")

In [200]:
corpus = []
for text in data.pq("LTTextLineHorizontal"):
    thing = str(text.layout)
    idx = thing.find("'")
    thing = thing[idx+1:-2]
    corpus.append(thing)

In [201]:
final_text = " ".join(i for i in corpus)

In [238]:
avoid = "§‡¶@∗"
symbols = "ΔΩπϴλθ°îĵk̂ûαβγδεζηθικλμνξοπρςστυφχψωΓΔΘΛΞΠΣΦΨΩϴ≤=∂γβα∈"
math = "≥∞<≤√=>+×"

In [203]:
#import string

#string.ascii_letters

In [204]:
#final_text = final_text.replace("\\n", "")

In [205]:
#if re.search("\d\.\d", c):
#    text = re.sub("\.", "~||~", s)

In [206]:
#final_text = re.sub("\[\d\]", "", final_text)
final_text = re.sub("al.", "al", final_text)
final_text = re.sub("\(cid:\d\d\d\)", "", final_text)
final_text = re.sub("\(cid:\d\d\)", "", final_text)
final_text = re.sub("\(cid:\d\)", "", final_text)

In [207]:
blob = TextBlob(final_text)
sentences = [str(sentence) for sentence in blob.sentences]

In [208]:
sentences = [s for s in sentences if len(s)>1]

In [213]:
len(sentences)

320

In [211]:
idx = 0
cnt = 0
for sent in sentences:
    if "References" in sent or "Appendix" in sent or "Bibliography" in sent or "REFERENCES" in sent or "acknowledgements" in sent or "ACKNOWLEDGEMENTS" in sent:
        idx = cnt
    else:
        cnt += 1

In [212]:
if idx > 0:
    sentences = sentences[:idx]

In [214]:
sentences = [s for s in sentences if not re.search('Fig', s)]

In [215]:
sentences = [s for s in sentences if len(s)>15]

In [216]:
no_symbols = []
cnt = 0
while cnt < len(sentences):
    counter = 0
    for symbol in symbols:
        if symbol in sentences[cnt]:
            counter += 1
            break
    if counter == 0:
        no_symbols.append(sentences[cnt])
    cnt += 1

In [217]:
no_math = []
cnt = 0
while cnt < len(no_symbols):
    counter = 0
    for m in math:
        if m in no_symbols[cnt]:
            counter += 1
            break
    if counter == 0:
        no_math.append(no_symbols[cnt])
    cnt += 1

In [218]:
no_avoid = []
cnt = 0
while cnt < len(no_math):
    counter = 0
    for n in avoid:
        if n in no_math[cnt]:
            counter += 1
            break
    if counter == 0:
        no_avoid.append(no_math[cnt])
    cnt += 1

In [219]:
len(no_avoid)

76

In [220]:
sentences = [s for s in no_avoid if len(s.split())>10]

In [222]:
sentences = [s for s in sentences if s[0].isupper() or s[0] in string.digits]

In [223]:
sentences = [s for s in sentences if s[-2:] != " ."]

In [233]:
sentences = [s.replace("\\n", "") for s in sentences]

In [239]:
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from wisdomaiengine import summarisepdfdocument

In [241]:
clean_text = " ".join(s for s in sentences)

In [243]:
summarisepdfdocument(clean_text)

['• The objective is to minimize the worst case expected detection time, relative to the distance of the hidden item to the origin.',
 '• A more tedious argument (and one of our technicaln contributions), as in the cow-path problem, shows that the aforementioned choice of geometricaly increasing xi’s for FSp is in fact optimalamong the family of monotone alorithms.',
 '• Given a robot’s trajectory T , probability p and distance d, the termination time ET (d) is deﬁned as the expected time that the robot detects the treasure for the ﬁrst time.',
 '• Somewhat surprisingly we show in Section 4 that the upper bound of Theorem 3.1 is in fact sub-optimal 3.1 An Upper Bound Using Monotone Trajectories In this section we propose a speciﬁc monotone alorithm with the performance promised by Theorem 3.1.',
 '• The vale of 3 is provably a lower bound to any search strategy since the searcher has to return at least once close to the origin before attempting for a second time an expansion of the sea